In [41]:
import requests
from bs4 import BeautifulSoup
import pymysql
import pandas as pd

In [42]:
data = requests.get('https://www.youm7.com/Section/اقتصاد-وبورصة/297/1').text

In [43]:
soup = BeautifulSoup(data)
soup.title.text

'اقتصاد وبورصة - اليوم السابع'

In [44]:
def Elyom7_details(url):
    data      = requests.get(url).text    
    soup      = BeautifulSoup(data)
    title     = soup.title.text
    image     = soup.find("div",attrs={'class':"img-cont"}).find('img')['src']
    keywords  = soup.find('meta',attrs={'name':'keywords'})['content']
    writer    = soup.find('div',attrs={'class':'articleCont'}).find('span').text
    article   = soup.find('div',attrs={'id':'articleBody'})
    body      = []
    for p in article.findAll('p'):
        if p.text != '':
            body.append(p.text)
    body      = ' '.join(body)
    return (url,title,keywords,image,writer,body)

In [45]:
Elyom7_details(" https://www.youm7.com/story/2021/5/3/193-مليون-جنيه-تراجع-بواردات-أسماك-الرنجة-بسبب-شهر-رمضان/5302258")

(' https://www.youm7.com/story/2021/5/3/193-مليون-جنيه-تراجع-بواردات-أسماك-الرنجة-بسبب-شهر-رمضان/5302258',
 '193 مليون جنيه تراجع بواردات أسماك الرنجة بسبب شهر رمضان - اليوم السابع',
 'واردات مصر,الرنجة,شم النسيم,اخبار الاقتصاد',
 'https://img.youm7.com/large/201906021116461646.jpg',
 'كتب: مدحت عادل',
 '\r\n\tيحتفل المصريون، غدا الاثنين بعيد شم النسيم، والذي يشهد عادة إقبالا كبيرا على استهلاك أسماك الرنجة، والتي تعتبر أحد أهم مظاهر الاحتفال بهذا العيد من كل عام. \r\n\tويتزامن الاحتفال بشم النسيم هذا العام خلال شهر رمضان، وهو ما ألقى بظلاله على واردات مصر الشهرية من الأسماك وخاصة الرنجة، حيث تتحرك واردات مصر من الرنجة ارتفاعا وانخفاضا تبعا لاحتياجات السوق المحلية قبلها بثلاثة أشهر على الأقل، وفي هذا الإطار يرصد الجهاز المركزي للتعبئة العامة والإحصاء، تراجع الواردات المصرية من الرنجة في شهر ديسمبر الماضي، فيما لم تسجل نشرة التجارة الشهرية التابعة للجهاز أية واردات للرنجة، ضمن قائمة أهم واردات السلع الغذائية في آخر إصدار لها في شهر يناير الماضي. \r\n\tوتراجعت واردات الرنجة، لتصل إلى 189 

In [6]:
soup = BeautifulSoup(data)
soup.title.text

'اقتصاد وبورصة - اليوم السابع'

In [109]:
def get_Elyom7_article_links(page):
    data = requests.get('https://www.youm7.com/Section/297/'+str(page)+'/اقتصاد-وبورصة').text   
    soup = BeautifulSoup(data) 
    

    for link in soup.find('div',attrs={'class' :'row news-content'}).findAll('a'):
        try:
            links = ["https://www.youm7.com/"+link.get('href')]
        except:
            pass
        #print(links)
        
    return str(page)+" ===> Done"   
  

In [110]:
get_Elyom7_article_links(1)

'1 ===> Done'

In [48]:
data = Elyom7_details('https://www.youm7.com/story/2021/5/4/تعرف-على-سعر-الدولار-مع-أول-يوم-عمل-للبنوك-بعد/5306181')
data

('https://www.youm7.com/story/2021/5/4/تعرف-على-سعر-الدولار-مع-أول-يوم-عمل-للبنوك-بعد/5306181',
 'تعرف على سعر الدولار مع أول يوم عمل للبنوك بعد الإجازة - اليوم السابع',
 'أسعار الدولار,سعر الدولار اليوم,أسعار الدولار اليوم,سعر الدولار اليوم فى مصر,الدولار اليوم,الدولار',
 'https://img.youm7.com/large/32016131843503491.jpg',
 'كتبت : سماح حمدى',
 '\r\n\t\t15.71 جنيه للبيع. \n \r\n\t\t15.71 جنيه للبيع. \n')

In [111]:
conn = pymysql.connect(host='localhost',port=3306,user='root',password='12345678')
cur  = conn.cursor()
query = "CREATE DATABASE IF NOT EXISTS crawler;"
cur.execute(query)
query = """
            create table  IF NOT EXISTS crawler.Elyom7(
                id int primary key auto_increment,
                url text,
                title text,
                keywords text,
                image text,
                writer text,
                body text
            ) DEFAULT CHARSET=utf8 DEFAULT COLLATE utf8_general_ci;
        """
cur.execute(query)
conn.commit()

In [112]:
def save_db(url,title,keywords,image,writer,body):
    query = """INSERT INTO crawler.Elyom7(url,title,keywords,image,writer,body) VALUES(%s,%s,%s,%s,%s,%s)"""
    val   = (url,title,keywords,image,writer,body)
    conn  = pymysql.connect(host='localhost',port=3306,user='root',password='12345678')
    cur   = conn.cursor()
    cur.execute(query,val)
    conn.commit()
    conn.close()
    return "Done"

In [113]:
get_Elyom7_article_links(1)

'1 ===> Done'

In [114]:
def get_Elyom7_details(url):
    
    data      = requests.get(url).text    
    soup      = BeautifulSoup(data)
    title     = soup.title.text
    image     = soup.find("div",attrs={'class':"img-cont"}).find('img')['src']
    keywords  = soup.find('meta',attrs={'name':'keywords'})['content']
    writer    = soup.find('div',attrs={'class':'articleCont'}).find('span').text
    article   = soup.find('div',attrs={'id':'articleBody'})
    body      = []
    for p in article.findAll('p'):
        if p.text != '':
            body.append(p.text)
    body      = ' '.join(body)
    save_db(url,title,keywords,image,writer,body)  
    return (url,title,keywords,image,writer,body)

In [115]:
for i in range(5,11):
    print(get_Elyom7_article_links(i))

5 ===> Done
6 ===> Done
7 ===> Done
8 ===> Done
9 ===> Done
10 ===> Done


In [116]:
import pandas as pd
conn = pymysql.connect(host='localhost',port=3306,user='root',password='12345678')
df = pd.read_sql_query("SELECT * FROM crawler.Elyom7",conn)
df.head()


,id,url,title,keywords,image,writer,body
0,1,https://www.youm7.com/story/2021/5/3/193-مليو...,193 مليون جنيه تراجع بواردات أسماك الرنجة بسبب...,"واردات مصر,الرنجة,شم النسيم,اخبار الاقتصاد",https://img.youm7.com/large/201906021116461646...,كتب: مدحت عادل,\r\n\tيحتفل المصريون، غدا الاثنين بعيد شم النس...
1,2,https://www.youm7.com/story/2021/5/3/193-مليو...,193 مليون جنيه تراجع بواردات أسماك الرنجة بسبب...,"واردات مصر,الرنجة,شم النسيم,اخبار الاقتصاد",https://img.youm7.com/large/201906021116461646...,كتب: مدحت عادل,\r\n\tيحتفل المصريون، غدا الاثنين بعيد شم النس...
2,3,https://www.youm7.com/story/2021/5/3/193-مليو...,193 مليون جنيه تراجع بواردات أسماك الرنجة بسبب...,"واردات مصر,الرنجة,شم النسيم,اخبار الاقتصاد",https://img.youm7.com/large/201906021116461646...,كتب: مدحت عادل,\r\n\tيحتفل المصريون، غدا الاثنين بعيد شم النس...
3,4,https://www.youm7.com/story/2021/5/3/193-مليو...,193 مليون جنيه تراجع بواردات أسماك الرنجة بسبب...,"واردات مصر,الرنجة,شم النسيم,اخبار الاقتصاد",https://img.youm7.com/large/201906021116461646...,كتب: مدحت عادل,\r\n\tيحتفل المصريون، غدا الاثنين بعيد شم النس...
4,5,https://www.youm7.com/story/2021/5/3/193-مليو...,193 مليون جنيه تراجع بواردات أسماك الرنجة بسبب...,"واردات مصر,الرنجة,شم النسيم,اخبار الاقتصاد",https://img.youm7.com/large/201906021116461646...,كتب: مدحت عادل,\r\n\tيحتفل المصريون، غدا الاثنين بعيد شم النس...
